# Compare exisiting LULC datasets

This notebook loads and examines available LULC datasets over SIDS countries.

Requirements for running this notebook:
* SIDS admin boundaries downloaded from GADM, saved in a data/gadm_boundaries folder. Total data size is ~100MB.
* Python environment with required packages including ODC.

Key datasets to expore from microsoft Planetary Computer (PC)
* ESA Climate Change Initiative (CCI) Land Cover (1992-2020, 300 m)
* ESA WorldCover (2020 and 2021, 10 m)
* Esri LULC (9-classes V2, 2017-2023, 10 m)


PC datasets can be listed from the [STAC viewer](https://radiantearth.github.io/stac-browser/#/external/planetarycomputer.microsoft.com/api/stac/v1/)


In [2]:
from pystac.client import Client
from odc.stac import load
import xarray as xr
from planetary_computer import sign_url

from odc.geo import BoundingBox

import sys, os
sys.path.append(os.path.abspath(".."))
from src.spatial import list_countries, get_country_boundary

In [3]:
# List SIDS countries
print(list_countries())

['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Cabo Verde', 'Comoros', 'Dominica', 'Fiji', 'Grenada', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Kiribati', 'Maldives', 'Marshall Islands', 'Mauritius', 'Micronesia (Federated States of)', 'Nauru', 'Palau', 'Papua New Guinea', 'Samoa', 'Sao Tome and Principe', 'Seychelles', 'Solomon Islands', 'St Kitts and Nevis', 'St Lucia', 'St Vincent and the Grenadines', 'Suriname', 'Timor-Leste', 'Tonga', 'Tuvalu', 'Vanuatu']


In [4]:
country = "Vanuatu"


In [5]:
gdf = get_country_boundary(country)
gdf

,GID_0,COUNTRY,geometry
0,VUT,Vanuatu,"MULTIPOLYGON (((169.77661 -20.24883, 169.77583..."


In [ ]:
# potentially buffer the bounding box to ensure full coverage
bbox = BoundingBox(*(gdf.total_bounds), crs="EPSG:4326")
bbox.explore()

In [8]:
# Use Planetary Computer STAC API
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/") 

In [ ]:
def load_lulc_data(product: str):
    items = catalog.search(
        collections=[product],
        intersects=bbox.polygon,
        datetime="2020-06",
    ).item_collection()

    print(f"Found {len(items)} items for product {product}")
    
    ds = load(
        items,
        intersects=bbox.polygon,
        crs="EPSG:6933",
        groupby="solar_day",
        resolution=100,
        chunks={"x": 2048, "y": 2048},
        patch_url=sign_url,
        resample_alg="nearest",
    )

    print(f"Product {product} has variables: {ds.data_vars}")
    
 return ds

# TODO
# Understand bands for each LULC dataset, including data quality bands
# Define mapping of different LULC classes
# Define how to compare 300 m with 10 m, resampling to 100m as a start


In [12]:
lulc_products = [
    "esa-cci-lc",
    "esa-worldcover",
    "io-lulc-annual-v02",    
]

datasets = []

for product in lulc_products:
    items = catalog.search(
        collections=[product],
        intersects=bbox.polygon,
        datetime="2020-06",
    ).item_collection()

    print(f"Found {len(items)} items for product {product}")
    
    ds = load(
        items,
        intersects=bbox.polygon,
        crs="EPSG:6933",
        groupby="solar_day",
        resolution=100,
        chunks={"x": 2048, "y": 2048},
        patch_url=sign_url,
        resample_alg="nearest",
    )

    print(f"Product {product} has variables: {ds.data_vars}")
    
    datasets.append(ds)

ds = xr.merge(datasets)
ds

Found 1 items for product esa-cci-lc
Product esa-cci-lc has variables: Data variables:
    lccs_class           (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    change_count         (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    processed_flag       (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    observation_count    (time, y, x) uint16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    current_pixel_state  (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Found 6 items for product esa-worldcover
Product esa-worldcover has variables: Data variables:
    map              (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    input_quality.1  (time, y, x) int16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    input_quality.2  (time, y, x) int16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarra

<xarray.Dataset> Size: 533MB
Dimensions:              (y: 8781, x: 3570, time: 1)
Coordinates:
  * y                    (y) float64 70kB -1.654e+06 -1.654e+06 ... -2.532e+06
  * x                    (x) float64 29kB 1.607e+07 1.607e+07 ... 1.643e+07
  * time                 (time) datetime64[ns] 8B 2020-01-01
    spatial_ref          int32 4B 6933
Data variables:
    lccs_class           (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    change_count         (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    processed_flag       (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    observation_count    (time, y, x) uint16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    current_pixel_state  (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    map                  (time, y, x) uint8 31MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    input_quality.1      (time, y, x) int16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    input_quality.2      (time, y, x) int16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    input_quality.3      (time, y, x) int16 63MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    data                 (time, y, x) float32 125MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [15]:
vars = list(ds.data_vars)

ds["esa_cci_lc"] = ds["lccs_class"]
ds["io_lulc"] = ds["data"]
ds["esa_worldcover"] = ds["map"]


new_ds = ds.drop_vars(vars)

data = new_ds.squeeze().load()

In [11]:
data.esa_worldcover.odc.explore()

In [14]:
data.io_lulc.odc.explore()

In [16]:
data.esa_cci_lc.odc.explore()